In [1]:
from torch.utils.data import DataLoader, random_split,TensorDataset
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import NN_classes
from torchvision import datasets, transforms
import training_methods
import DataLoading



In [2]:
#executions = ["Network_Line_Out_N_101_N_102_cac1","Network_Line_Out_N_102_N_104_cac1","Network_Line_Out_N_101_N_105_cac1","Network_Line_Out_N_102_N_104_cac1","Network_Line_Out_N_102_N_106_cac1","Network_Line_Out_N_103_N_109_cac1"]
#executions = ["Network_Line_Out_N_101_N_102_cac1"]
#executions = ["Network_Full_Generation_Full","Network_Line_In_N_101_N_102_cac1","Network_Line_In_N_101_N_103_cac1","Network_Line_In_N_101_N_105_cac1"]

executions = ["Network_Full_Generation_Full"]
sc = "sc01"
period = "2030"
folder = ""
te_s = 0.1
val_s = 0.2

In [3]:
dfs_in,dfs_out = DataLoading.load_data(folder,executions,period,sc)

Data/input_f_sc01_Network_Full_Generation_Full_2030.csv
28


In [4]:
ts_in,ts_out =  DataLoading.split_tr_val_te(dfs_in,dfs_out,executions,te_s,val_s)

In [5]:
d_ft_in, d_ft_out = DataLoading.concat_and_normalize(ts_in,ts_out,executions)

In [6]:
train = TensorDataset(d_ft_in['train'].float(), d_ft_out['train'].float())
validation = TensorDataset(d_ft_in['val'].float(), d_ft_out['val'].float())

training_loader = DataLoader(train,batch_size=32)
validation_loader = DataLoader(train,batch_size=32)

In [7]:
d_ft_in['train'].shape

torch.Size([6289, 28])

In [15]:
learning_rates = [0.01*4**i for i in range(4)]
loss_fn = torch.nn.MSELoss()
nbs_e = [4,8]
i=0
nbs_hidden = [0,3]
results = pd.DataFrame()
for nb_e in nbs_e:
    for lr in learning_rates:
        for nb_hidden in nbs_hidden: 
            if nb_hidden == 0: 
                m = NN_classes.ObjectiveEstimator_ANN_Single_layer(input_size=d_ft_in['train'].shape[1],output_size=1)
                
            elif nb_hidden == 3: 
                m = NN_classes.ObjectiveEstimator_ANN_3hidden_layer(input_size=d_ft_in['train'].shape[1],hidden_size1=int(d_ft_in['train'].shape[1]/4),hidden_size2=int(d_ft_in['train'].shape[1]/16),hidden_size3=int(d_ft_in['train'].shape[1]/64),output_size=1)
            m_name = f"OE_{nb_hidden}h_{nb_e}e_{lr}lr"
            optimizer = torch.optim.Adam(m.parameters(), lr=lr)
            train_loss = training_methods.train_multiple_epochs(nb_e,m,training_loader,validation_loader,loss_fn,optimizer,m_name,True)
            
            test_predictions = m(d_ft_in["test"].float())
            test_loss = loss_fn(test_predictions,d_ft_out["test"])
            train_predictions = m(d_ft_in["train"].float())
            train_loss = loss_fn(train_predictions,d_ft_out["train"])
            
            r = pd.DataFrame({"Model_type": m_name,"Epochs": nb_e,"Lr":lr, "Tr_l":train_loss.item(),"Te_l":test_loss.item()},index = [i]
            )
            i+=1
            results = pd.concat([results,r])

EPOCH 1:
  batch 1 loss: 0.0015155567228794097
  batch 101 loss: 0.004074039371880644


C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


LOSS train 0.0029265735061363295 valid 0.00022116019681561738


RuntimeError: Parent directory trained_models/min_val does not exist.

In [11]:
results

,Model_type,Epochs,Lr,Tr_l,Te_l
0,sl,4,0.01,1.995044e-02,3.113820e-02
1,3h,4,0.01,6.880393e-08,5.301107e-08
2,sl,4,0.04,2.364681e-04,2.971305e-04
3,3h,4,0.04,6.932472e-08,5.343779e-08
4,sl,4,0.16,2.749902e-05,2.335273e-05
5,3h,4,0.16,6.936998e-08,5.347490e-08
6,sl,4,0.64,2.291427e-04,1.987444e-04
7,3h,4,0.64,6.930760e-08,5.342376e-08
8,sl,8,0.01,2.233874e-06,1.735164e-06
9,3h,8,0.01,2.290619e-08,3.574458e-08
